# Niklas Goes BERT
---------------
``` 



In [1]:
#import modules
import numpy as np 
import pandas as pd 
import random
import time
import csv
import nltk
import pickle
import time
import gc
import os
import torch
from datasets import Dataset
from tqdm.auto import tqdm
nltk.download('punkt')

print(torch.__version__)

/Users/niklashahn/anaconda3/envs/Masterarbeit_Speeches/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.2.0


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/niklashahn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Wrangling

In [13]:
spgl_dat = pd.read_csv("/Users/niklashahn/Library/CloudStorage/Dropbox/Masterarbeit/Daten_Analyse/_data/_processed/sample_classification/sampling_data_media.csv")
#spgl_dat = spgl_dat.dropna(subset=['description'])
spgl_dat = spgl_dat.dropna(subset=['description'])

hg_data = Dataset.from_pandas(spgl_dat)
hg_data
#dataset = hg_data.map(tokenize_dataset, batched=True, remove_columns=hg_data.column_names)


/Users/niklashahn/anaconda3/envs/Masterarbeit_Speeches/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


Dataset({
    features: ['description', 'newspaper', 'id'],
    num_rows: 100
})

In [3]:
spgl_dat["description"][118766]# desc = haedline

'Bruce Lee machte das Martial-Arts-Kino aus Hongkong weltberühmt. Nun will der oscarprämierte Regisseur Ang Lee das Leben dieses »brillanten« Mannes verfilmen – mit seinem Sohn in der Hauptrolle.'

Loading BERT
--------------------


model

In [14]:
from transformers import (AutoModelForSequenceClassification, AutoTokenizer, 
                          Trainer, TrainingArguments)

CAP_NUM_DICT = {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 
6: '7', 7: '8', 8: '9', 9: '10', 10: '12', 11: '13', 12: '14', 
13: '15', 14: '16', 15: '17', 16: '18', 17: '19', 18: '20', 19: 
'21', 20: '23', 21: '999'}
num_labels = len(CAP_NUM_DICT)


model = AutoModelForSequenceClassification.from_pretrained('poltextlab/xlm-roberta-large-german-media-cap-v3',
                                                           num_labels=num_labels,
                                                           problem_type="multi_label_classification",
                                                           ignore_mismatched_sizes=True
                                                           )
device = torch.device("mps")
model.to("mps")

#training_args = TrainingArguments(
#    output_dir=".",
#    logging_strategy='epoch',
#    num_train_epochs=10,
#    per_device_train_batch_size=8,
#    per_device_eval_batch_size=8,
#    learning_rate=5e-06,
#    seed=42,
#    save_strategy='epoch',
#    evaluation_strategy='epoch',
#    save_total_limit=1,
#    load_best_model_at_end=True
#)

training_args = TrainingArguments(
    output_dir='.',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8
)

trainer = Trainer(
    model=model,
    args=training_args
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at poltextlab/xlm-roberta-large-german-media-cap-v3 and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([21, 1024]) in the checkpoint and torch.Size([22, 1024]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([21]) in the checkpoint and torch.Size([22]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenize

In [15]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')

def tokenize_dataset(data : pd.DataFrame):
    #tokenized = tokenizer(data["description"],
    tokenized = tokenizer(data["description"],
                          max_length=128,
                          truncation=True,
                          padding="max_length")
    return tokenized


dataset = hg_data.map(tokenize_dataset, batched=True, remove_columns=hg_data.column_names)

In [16]:
first_item = dataset[0]

# Print the keys
print(dir(first_item))

['__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']


In [17]:
probs = trainer.predict(test_dataset=dataset).predictions
predicted = pd.DataFrame(np.argmax(probs, axis=1)).replace({0: CAP_NUM_DICT}).rename(
    columns={0: 'predicted'}).reset_index(drop=True)


100%|██████████| 13/13 [00:01<00:00,  6.66it/s]


In [18]:
spgl_dat_class = pd.concat([spgl_dat, predicted], axis=1)

In [19]:
print(spgl_dat_class.head())
spgl_dat_class.to_csv('_data/_processed/sample_classification/bert_media.csv', index=False)

                                         description newspaper  id predicted
0  Hier finden Sie Informationen zu dem Thema „Kr...      zeit   1         5
1  Hier finden Sie Informationen zu dem Thema „Ge...      zeit   2        14
2  Im März kritisierte Lise Klaveness Katar auf o...   spiegel   3         4
3  Adler-Antrag auf Insolvenz in Eigenverwaltung ...      welt   4         4
4  Germany has made massive changes to foreign an...   spiegel   5         5


# Old BERT Stuff For Reference - Likely Unncessary

### Preprocess Test Data

In [ ]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

def preproc_df(dataframe):

    test_content = tokenizer(dataframe['tweet'].tolist(), padding= "max_length" , truncation=True, return_tensors="pt")
    test_labels = dataframe['label'].map({"sexist": 1, "non-sexist": 0}).tolist()

    # Assuming 'test_df' is your DataFrame with 'text' column for test tweets and 'label' column for test labels
    # Preprocess data (tokenize, encode) and convert labels to numerical form if needed

    # Tokenize input text for test data
    class TweetDataset(Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

    # Create a test dataset
    test_dataset = TweetDataset(test_content, test_labels)  # Assuming TweetDataset is similar to the one used in training

    # Create a DataLoader for test data
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    return test_loader

As for part 1.1., the models are tested on the whole dataset and the individual datasets

### Test Function

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_model(model, test_loader):
    model.eval()
    predictions = []
    true_labels = []
    
    
    device = torch.device("cuda")
    model.to("cuda")

    with torch.no_grad():
        for batch in test_loader:
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)

            outputs = model(**inputs)
            logits = outputs.logits
            _, predicted = torch.max(logits, 1)

            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    confusion = confusion_matrix(true_labels, predictions)

    #print(f"Accuracy: {accuracy:.4f}")
    #print(f"Precision: {precision:.4f}")
    #print(f"Recall: {recall:.4f}")
    #print(f"F1-score: {f1:.4f}")
    #print("Confusion Matrix:")
    #print(confusion)
    
    return f1

# Usage:
# Assuming you have a test_loader for your test data (similar to the train_loader used during training)
# Call evaluate_model function after training to evaluate the model's performance
# For example:
# evaluate_model(model, test_loader)


In [ ]:
f1_macro = evaluate_model(bert_m1, bert_test_ood1)
f1_macro

In [ ]:
f1_scores_bert = []
datasets = ["ood1", "ood2", "ood3", "hc", "full"]
models = ["m1", "m2", "m3_gpt", "m3_flan", "m4_gpt", "m4_flan"]

for model in models:
    current_model = load_bert_model(f"/kaggle/input/bert-outputs/bert_outputs/{model}")
        
    for df in datasets:
        df_name = f"bert_test_{df}"
        current_df = globals()[df_name]  
        
        f1_macro = evaluate_model(current_model, current_df)
        f1_scores_bert.append(f1_macro)

In [ ]:
f1_scores_bert

In [ ]:
data_array = [f1_scores_bert[i:i+5] for i in range(0, len(f1_scores_bert), 5)]
f1_scores_bert_df = pd.DataFrame(data_array, columns=["ood1", "ood2", "ood3", "hc", "full"], index=["og",  "mcad", "acad_gpt", "acad_flan", "aex_gpt", "aex_flan"])

print(f1_scores_bert_df)

f1_scores_bert_df.to_csv(f"f1_scores_bert_df.csv", index=False)


In [ ]:
f1_scores_bert_df = pd.read_csv('/kaggle/input/outputs-all/f1_scores_bert_df.csv')
f1_scores_bert_df

In [ ]:
f1_scores_svm_df.to_csv(f"f1_scores_svm_df.csv", index=False)


I write my own Bert Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
num_labels = len(CAP_NUM_DICT)